# Learning and Decision Making

## Laboratory 6: Reinforcement learning

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. The windy gridworld domain

Consider the larger version of the windy gridworld domain depicted in the figure below.

<img src="windy.png" width="400px">

In it, a boat must navigate a 7 &times; 10 gridworld, to reach the goal cell, marked with _G_. There is a crosswind upward through the middle of the grid, in the direction indicated by the gray arrows. The boat has available the standard four actions -- _Up_, _Down_, _Left_ and _Right_. In the region affected by the wind, however, the resulting next state is shifted upward as a consequence of the crosswind, the strength of which varies from column to column. The strength of the wind is given below each column, and corresponds to the number of cells that the movement is shifted upward. For example, if the boat is one cell to the right of the goal, then the action _Left_ takes you to the cell just above the goal.

The agent pays a cost of 1 in every step before reaching the goal. The problem can be described as an MDP $(\mathcal{X},\mathcal{A},\mathbf{P},c,\gamma)$ as follows.

In [1]:
%matplotlib notebook
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

np.set_printoptions(threshold=10)

# Problem specific parameters
WIND = (0, 0, 0, 1, 1, 1, 2, 2, 1, 0)
nrows = 7
ncols = 10
init = [3, 0]
goal = [3, 7]

# States
X = [[x, y] for x in range(nrows) for y in range(ncols)]
nX = len(X)

# Actions
A = ['U', 'D', 'L', 'R']
nA = len(A)

# Transition probabilities
P = dict()
P['U'] = np.zeros((nX, nX))
P['D'] = np.zeros((nX, nX))
P['L'] = np.zeros((nX, nX))
P['R'] = np.zeros((nX, nX))

for i in range(len(X)):
    x = X[i]
    y = dict()
    
    
    y['U'] = [x[0] - WIND[x[1]] - 1, x[1]]
    y['D'] = [x[0] - WIND[x[1]] + 1, x[1]]
    y['L'] = [x[0] - WIND[x[1]], x[1] - 1]
    y['R'] = [x[0] - WIND[x[1]], x[1] + 1]
    
    for k in y:
        y[k][0] = max(min(y[k][0], nrows - 1), 0)
        y[k][1] = max(min(y[k][1], ncols - 1), 0)
        j = X.index(y[k])
        P[k][i, j] = 1

c = np.ones((nX, nA))
c[X.index(goal), :] = 0

gamma = 0.99

# -- Pretty print

print('\n- MDP problem specification: -\n')

print('States:')
print(np.array(X))

print('\nActions:')
print(A)

print('\nTransition probabilities:')
for a in A:
    print('Action', a)
    print(P[a])
    
print('\ncost:')
print(c)

print('\nStart state:', init)
print('\nGoal state:', goal)


- MDP problem specification: -

States:
[[0 0]
 [0 1]
 [0 2]
 ..., 
 [6 7]
 [6 8]
 [6 9]]

Actions:
['U', 'D', 'L', 'R']

Transition probabilities:
('Action', 'U')
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
('Action', 'D')
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
('Action', 'L')
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
('Action', 'R')
[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  

---

#### Activity 1.        

Compute the optimal _Q_-function for the MDP defined above using value iteration. As your stopping condition, use an error between iterations smaller than `1e-8`.

---

In [2]:
# Insert your code here.

import numpy as np 

i = 0

Q_Optimal = np.zeros((70,4))
   
err = 1

# Q-Function computation

#while i < 10:
while err > 1e-8:

    Q_Up = c[:,0][np.newaxis, :].T + gamma * P["U"].dot(Q_Optimal)
    Q_Down = c[:,1][np.newaxis, :].T + gamma * P["D"].dot(Q_Optimal)
    Q_Left = c[:,2][np.newaxis, :].T + gamma * P["L"].dot(Q_Optimal)
    Q_Right = c[:,3][np.newaxis, :].T + gamma * P["R"].dot(Q_Optimal)
        
    Qnew = np.min((Q_Up, Q_Down, Q_Left, Q_Right), axis=0)

    err = np.linalg.norm(Qnew - Q_Optimal)
        

    Q_Optimal = Qnew
        

    i += 1
        

print "Iterations:", i
print Q_Optimal





Iterations: 2107
[[ 88.86732306  88.86732306  88.86732306  88.86732306]
 [ 88.75487178  88.75487178  88.75487178  88.75487178]
 [ 88.64128462  88.64128462  88.64128462  88.64128462]
 ..., 
 [ 87.18534608  87.18534608  87.18534608  87.18534608]
 [ 87.31349262  87.31349262  87.31349262  87.31349262]
 [ 87.44035769  87.44035769  87.44035769  87.44035769]]


---

#### Activity 2.        

Write down a Python function that, given a Q-function $Q$ and a state $x$, selects a random action using the $\epsilon$-greedy policy obtained from $Q$ for state $x$. Your function should receive an optional parameter, corresponding to $\epsilon$, with default value of 0.1. 

**Note:** In the case of two actions with the same value, your $\epsilon$-greedy policy should randomize between the two.

---

In [3]:
# Insert your code here.
import numpy as np
import random
#t = np.array([[1,1,1]])

def select_action(Q,nX,e_value = None):
    if e_value is None:
        e_value = 0.1
    
    #print "e:", e_value
    
    state_index = X.index(nX)

    # primeiro verifica qual é o min da lista depois verifica se existem mais do que 1 min
    # nesse caso vai guardar os indices desses valores para depois escolher um aleatoriamente
    
    aux=[]
    for i,e in enumerate(Q[state_index]):
        if e == np.amin(Q[state_index]):
            aux += [i] 

    
    if len(aux)>1:
        action_index = random.choice(aux)
    else:
        action_index = aux[0]      
    
    
    # pega no indice obtido e escolhe a acçao correspondente
    selected_action = A[action_index]
    
    return selected_action

    
select_action(Q_Optimal,[0,0])

'D'

### 2. Model-based learning

You will now run the model-based learning algorithm discussed in class, and evaluate its learning performance.

# ---

#### Activity 3.        

Run the model-based reinforcement learning algorithm discussed in class to compute $Q^*$ for 100,000 iterations. Initialize each transition probability matrix as the identity and the cost function as all-zeros. Use an $\epsilon$-greedy policy with $\epsilon=0.1$ (use the function from Activity 2). Note that, at each step,

* You will need to select an action according to the $\epsilon$-greedy policy;
* The state and action, you will then compute the cost and generate the next state; 
* With this transition information (state, action, cost, next-state), you can now perform an update. 
* When updating the components $(x,a)$ of the model, use the step-size

$$\alpha_t=\frac{1}{N_t(x,a)+1},$$

where $N_t(x,a)$ is the number of visits to the pair $(x,a)$ up to time step $t$.

In order to ensure that your algorithm visits every state and action a sufficient number of times, after the boat reaches the goal cell, make one further step, the corresponding update, and then reset the position of the boat to a random state in the environment.

Plot the norm $\|Q^*-Q^{(k)}\|$ every 500 iterations of your method, where $Q^*$ is the optimal _Q_~function computed in Activity 1.

**Note:** The simulation may take a bit. Don't despair.

---

In [8]:
# Insert your code here.

# Inicializar probabilidades de transiçao a 0
P_Up = np.eye(nX, nX)
P_Down = np.eye(nX, nX)
P_Left = np.eye(nX, nX)
P_Right = np.eye(nX, nX)

# Inicializar custos a 0
cost = np.zeros((nX, nA))

state_visits = np.zeros((70,4))


def calc_step_size(curent_state, action):
    return 1 / ( state_visits[X.index(current_state), A.index(action)] + 1 )
    

def update_cost(current_state):
    
    #ct+1(xt,at)= Mct(xt,at)+alpha_t(ct - Mct(xt,at))
    
    #new_cost = ? + step_size*(cost[current_state] - ?)
    
    if current_state == goal:
        cost[X.index(current_state), :] = 0
    else:
        cost[X.index(current_state), :] = 1

def update_state(current_state, action):
    
# AS COORDENADAS DO PROF ESTAO COMO [Y,X] for some reason ...

    if action == "U":
        
        if WIND[current_state[1]]==0:
            new_state = [current_state[0] + 1, current_state[1]]
        elif WIND[current_state[1]]==1:
            new_state = [current_state[0] + 2, current_state[1]]
        else:
            new_state = [current_state[0] + 3, current_state[1]]
        
        if new_state[0]>6:
            new_state[0]=6
            
        
    elif action == "D":
        
        if WIND[current_state[1]]==0:
            new_state = [current_state[0] - 1, current_state[1]]
        elif WIND[current_state[1]]==1:    
            new_state = [current_state[0], current_state[1]]
        else:
            new_state = [current_state[0] + 1, current_state[1]]
        
        if new_state[0]>6:
            new_state[0]=6
        if new_state[0]<0:
            new_state[0]=0
            
    elif action == "L":
      
        if WIND[current_state[1]]==0:
            new_state = [current_state[0], current_state[1] - 1]
        elif WIND[current_state[1]]==1:
            new_state = [current_state[0] + 1, current_state[1] - 1]
        else:
            new_state = [current_state[0] + 2, current_state[1] - 1]
            
        if new_state[0]>6:
            new_state[0]=6
        if new_state[1]<0:
            new_state[1]=0
        
    elif action == "R":
        if WIND[current_state[1]]==0:
            new_state = [current_state[0], current_state[1] + 1]
        elif WIND[current_state[1]]==1:
            new_state = [current_state[0] + 1, current_state[1] + 1]
        else:
            new_state = [current_state[0] + 2, current_state[1] + 1]
            
        if new_state[0]>6:
            new_state[0]=6
        if new_state[1]>9:
            new_state[1]=9
    
    # state_visits[estado, acçao]
    
    #print "next state antes do if:", new_state
    

    return new_state
    
    


#############################
#############################

i = 0

Q_k = np.zeros((70,4))

goal_reached = False

current_state = [0, 0] # TEMOS DE SUBSTITUIR PELO ESTADO INICIAL !!!

while i < 1000: # demora uns 30 segundos a correr 100000 iteraçoes

    Q_Up = cost[:,0][np.newaxis, :].T + gamma * P_Up.dot(Q_k)
    Q_Down = cost[:,1][np.newaxis, :].T + gamma * P_Down.dot(Q_k)
    Q_Left = cost[:,2][np.newaxis, :].T + gamma * P_Left.dot(Q_k)
    Q_Right = cost[:,3][np.newaxis, :].T + gamma * P_Right.dot(Q_k)
        
    Qnew = np.min((Q_Up, Q_Down, Q_Left, Q_Right), axis=0)
        
    Q_k = Qnew
    
    
    if i%500 == 0:
        print "Norm:", np.linalg.norm(Q_Optimal - Q_k)
    
    
    print "------------------------------------"
    print "iteration:",i
    print "current state:", current_state
    

    update_cost(current_state)
    
    if goal_reached == True:
        
        print "\n JA PASSAMOS PELO GOAL \n"
        
        next_state = random.choice(X)
        
        state_visits[X.index(current_state), A.index(action)] += 1
    
        #print state_visits
    
        step_size = calc_step_size(current_state, action)
    
        print "next state:", next_state
        
        current_state = next_state
        
    else:
        
        action = select_action(Q_k,current_state)

        print "action:", action
        #print "cost",current_state, "-->", cost[X.index(current_state), :] 

        next_state = update_state(current_state, action)

        #print "next state fora do update:", next_state

        state_visits[X.index(current_state), A.index(action)] += 1

        #print state_visits

        step_size = calc_step_size(current_state, action)


        print "next state:", next_state
        #print "step size:", step_size
        #print state_visits
        #print "\n"

        current_state = next_state  
    
    if current_state == goal:
        goal_reached = True
        
        
    i += 1
        

print "Iterations:", i
print "done"





Norm: 1475.81408634
------------------------------------
iteration: 0
current state: [3, 0]
action: R
next state: [3, 1]
------------------------------------
iteration: 1
current state: [3, 1]
action: D
next state: [2, 1]
------------------------------------
iteration: 2
current state: [2, 1]
action: L
next state: [2, 0]
------------------------------------
iteration: 3
current state: [2, 0]
action: U
next state: [3, 0]
------------------------------------
iteration: 4
current state: [3, 0]
action: U
next state: [4, 0]
------------------------------------
iteration: 5
current state: [4, 0]
action: D
next state: [3, 0]
------------------------------------
iteration: 6
current state: [3, 0]
action: L
next state: [3, 0]
------------------------------------
iteration: 7
current state: [3, 0]
action: U
next state: [4, 0]
------------------------------------
iteration: 8
current state: [4, 0]
action: L
next state: [4, 0]
------------------------------------
iteration: 9
current state: [4, 0]

### 3. Temporal-difference learning

You will now run both Q-learning and SARSA, and compare their learning performance with that of the model-based method just studied.

---

#### Activity 4.        

Repeat Activity 3 but using the _Q_-learning algorithm with a learning rate $\alpha=0.3$.

---

In [138]:
# Insert your code here.

# Inicializar probabilidades de transiçao a 0
P_Up = np.eye(nX, nX)
P_Down = np.eye(nX, nX)
P_Left = np.eye(nX, nX)
P_Right = np.eye(nX, nX)

# Inicializar custos a 0
cost = np.zeros((nX, nA))

state_visits = np.zeros((70,4))

i = 0

goal_reached = False

current_state = [0, 0] # TEMOS DE SUBSTITUIR PELO ESTADO INICIAL !!!

while i < 1000: # demora uns 30 segundos a correr 100000 iteraçoes
    
    # Qtx+1(Xt,At) = Qt(Xt,At) + alpha * [Ct + gamma*min(Qt(Xt+1,a')) - Qt(Xt,At)]
    
    



---

#### Activity 5.

Repeat Activity 4 but using the SARSA algorithm.

---

In [188]:
# Insert your code here.


[3]
3


---

#### Activity 6.

Discuss the differences observed between the performance of the three methods.

---

_Add your discussion here._